In [1]:
from datetime import datetime
import logging
import pandas as pd
import random
from requests_html import HTML
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service
from selenium.common.exceptions import *
from time import sleep

In [2]:
logging.basicConfig(format='%(levelname)s | %(asctime)s | %(message)s')
logger = logging.getLogger()
logger.setLevel(logging.INFO)

service = Service('geckodriver.exe')
options = webdriver.FirefoxOptions()
# options.add_argument('-headless')

base_url = 'https://www.beckershospitalreview.com'
# url = 'https://www.beckershospitalreview.com/pharmacy.html'
# site_channel = 'pharmacy'
# url = 'https://www.beckershospitalreview.com/legal-regulatory-issues.html'
# site_channel = 'legal-regulatory-issues'
# url = 'https://www.beckershospitalreview.com/hospital-physician-relationships.html'
# site_channel = 'physician'
# url = 'https://www.beckershospitalreview.com/hospital-management-administration.html'
# site_channel = 'leadership'
# url = 'https://www.beckershospitalreview.com/strategy.html'
# site_channel = 'strategy'
# url = 'https://www.beckershospitalreview.com/cardiology.html'
# site_channel = 'cardiology'
# url = 'https://www.beckershospitalreview.com/oncology.html'
# site_channel = 'oncology'
url = 'https://www.beckershospitalreview.com/hospital-transactions-and-valuation.html'
site_channel = 'transaction-valuation'

# base_url = 'https://www.beckerspayer.com'
# url = 'https://www.beckerspayer.com'
# site_channel = 'payer'

driver = webdriver.Firefox(service=service, options=options)
# wait = WebDriverWait(driver, 10)


urls = ['https://www.beckershospitalreview.com/pharmacy.html',
        'https://www.beckershospitalreview.com/legal-regulatory-issues.html', 
        'https://www.beckershospitalreview.com/hospital-physician-relationships.html',
        'https://www.beckershospitalreview.com/hospital-management-administration.html',
        'https://www.beckershospitalreview.com/strategy.html',
        'https://www.beckershospitalreview.com/cardiology.html',
        'https://www.beckershospitalreview.com/oncology.html']

driver.get(url)
logging.info(f'Opening {url}...')
sleep(15)

article_list = []
for page in range(10):
    logging.info(f'Loading page {page}')
    sleep(random.randint(5, 10))
    logging.info('Checking for pop up...')
    try:
        close_popup = driver.find_element(By.CSS_SELECTOR, '.olyticsBandBR')
        logging.info('Pop up found...closing it.')
        close_popup.click()
    except NoSuchElementException:
        logging.info('No pop up detected. Continuing.')
    try:
        close_newsletter_popup = driver.find_element(By.CSS_SELECTOR, '.fancybox-item')
        logging.info('Pop up found...closing it.')
        close_newsletter_popup.click()
    except NoSuchElementException:
        logging.info('No pop up detected. Continuing.')
    

    body = driver.page_source
    html = HTML(html=body)
    articles = html.find('.article')

    logging.info('Getting articles and appending to list.')
    for article in articles:
        try:
            title = article.find('.article-title', first=True).text
    #         print(title)
        except:
            None
        try:
            link = f'{base_url}' + article.find('.article-title', first=True).links.pop()
    #         print(link)
        except:
            None
        try:
            date = article.find('.article-date > time', first=True)
            date = date.attrs['datetime']
            date = date.split('T')[0]
    #         print(date)
        except:
            None
        try:
            channel = link.split('/')[3]
    #         print(channel)
        except:
            None
            
        data = {'date': date,
                'source': url,
                'channel': channel,
                'title': title,
                'link': link}
        
        article_list.append(data)

    sleep(random.randint(5, 10))

    try:
        driver.find_element(By.LINK_TEXT, 'Next').click()
    except ElementClickInterceptedException:
        driver.find_element(By.LINK_TEXT, 'Next').click()

logging.info('Compiling all articles into Excel file.')
df = pd.DataFrame(article_list)
df.channel = df.channel.str.lower()
df.title = df.title.str.lower()
df.link = df.link.str.lower()
df = df[['date', 'source', 'channel', 'title', 'link']]
df.drop_duplicates(inplace=True)
datestamp = datetime.today().strftime('%Y%m%dT%H%M')
df.to_excel(f'Beckers_Hospital_Review_Articles_{site_channel}_{datestamp}.xlsx', index=False, encoding='utf-8')
logging.info(f'Excel file Beckers_Hospital_Review_Articles_{site_channel}_{datestamp}.xlsx created.')

INFO | 2022-04-14 15:22:23,962 | Opening https://www.beckershospitalreview.com/hospital-transactions-and-valuation.html...
INFO | 2022-04-14 15:22:38,971 | Loading page 0
INFO | 2022-04-14 15:22:45,979 | Checking for pop up...
INFO | 2022-04-14 15:22:46,027 | Pop up found...closing it.
INFO | 2022-04-14 15:22:46,301 | Pop up found...closing it.
INFO | 2022-04-14 15:22:46,652 | Getting articles and appending to list.
INFO | 2022-04-14 15:22:57,204 | Loading page 1
INFO | 2022-04-14 15:23:05,211 | Checking for pop up...
INFO | 2022-04-14 15:23:05,222 | No pop up detected. Continuing.
INFO | 2022-04-14 15:23:05,232 | No pop up detected. Continuing.
INFO | 2022-04-14 15:23:05,348 | Getting articles and appending to list.
INFO | 2022-04-14 15:23:15,725 | Loading page 2
INFO | 2022-04-14 15:23:22,731 | Checking for pop up...
INFO | 2022-04-14 15:23:22,740 | No pop up detected. Continuing.
INFO | 2022-04-14 15:23:22,749 | No pop up detected. Continuing.
INFO | 2022-04-14 15:23:22,841 | Gettin